# Métodos iterativos

## Introducción

Hasta ahora sólo hemos visto métodos de solución directa. La característica común de estos métodos es que calculan la solución con un número finito de operaciones. Además, si el ordenador tuviera una precisión infinita (sin errores de redondeo), la solución sería exacta.
Los métodos iterativos, o indirectos, comienzan con una estimación inicial de la solución $\mathbf{x}$ y van mejorándola progresivamente hasta que el cambio en $\mathbf{x}$ es insignificante. Dado que el número de iteraciones necesario puede ser elevado, los métodos indirectos son, en general, más lentos que los directos. Sin embargo, los métodos iterativos tienen las siguientes dos ventajas que los hacen convenientes para ciertos problemas:

1. Es posible almacenar sólo los elementos no nulos de la matriz de coeficientes. Esto permite tratar con matrices muy grandes que sean dispersas (y no necesariamente por bandas). En muchos problemas, no es ni siquiera necesario almacenar la matriz de coeficientes.
2. Los procedimientos iterativos se *autocorrigen*. Esto significa que los errores de redondeo (o incluso los errores aritméticos) en una determinada iteración se corrigen en las iteraciones subsiguientes.

Un grave inconveniente de los métodos iterativos es que no siempre convergen a la solución. Se puede demostrar que la convergencia está garantizada solo si la matriz de coeficientes es diagonal dominante. El valor inicial de $\mathbf{x}$ no determina en ningún caso si el algoritmo convergerá o no: si el método converge para un vector inicial, lo hará para cualquiera. El valor inicial sólo afecta al número de iteraciones necesarias para la convergencia.

## Método Gauss-Seidel

El sistema de ecuaciones $\mathbf{Ax} = \mathbf{b}$ puede escribirse en notación escalar como:

$$
\sum_{j=1}^n A_{i j} x_j=b_i, \quad i=1,2, \ldots, n
$$

Extrayendo el término que contiene $x_i$ del sumatorio se obtiene

$$
A_{i i} x_i+\sum_{\substack{j=1 \\ j \neq i}}^n A_{i j} x_j=b_i, \quad i=1,2, \ldots, n
$$

Resolviendo para $x_i$ obtenemos

$$
x_i=\frac{1}{A_{i i}}\left(b_i-\sum_{\substack{j=1 \\ j \neq i}}^n A_{i j} x_j\right), \quad i=1,2, \ldots, n
$$

Esta última ecuación sugiere el siguiente esquema iterativo:
$$
x_i \leftarrow \frac{1}{A_{i i}}\left(b_i-\sum_{\substack{j=1 \\ j \neq i}}^n A_{i j} x_j\right), \quad i=1,2, \ldots, n
$$
 
Comenzamos eligiendo el vector inicial $\mathbf{x}$. Si no se dispone de una buena estimación de la solución, $\mathbf{x}$ puede elegirse al azar. A continuación, se utiliza la ecuación anterior para volver a calcular cada elemento de $\mathbf{x}$, utilizando siempre los últimos valores disponibles de $x_j$. El procedimiento se repite hasta que los cambios en $\mathbf{x}$ entre las sucesivas iteraciones sean lo suficientemente pequeños.

La convergencia del método Gauss-Seidel puede mejorarse mediante una técnica conocida como *relajación*. La idea es tomar el nuevo valor de $x_i$ como una media ponderada de su valor anterior y el nuevo. La fórmula iterativa correspondiente es

$$
x_i \leftarrow \frac{\omega}{A_{i i}}\left(b_i-\sum_{\substack{j=1 \\ j \neq i}}^n A_{i j} x_j\right)+(1-\omega) x_i, \quad i=1,2, \ldots, n
$$ 

donde el peso $\omega$ se llama factor de relajación. Se puede ver que si $\omega = 1$, no se produce ninguna relajación. Si $\omega < 1$, la ecuación anterior representa una interpolación entre la antigua $x_i$ y la nueva. Esto se denomina *infra-relajación*. En los casos en los que $\omega > 1$, tenemos extrapolación, o *sobrerrelajación*.

No existe un método inequívoco para determinar de antemano el valor óptimo de $\omega$, lo que suele hacerse es calcular una estimación al inicio del proceso iterativo. Sea $\Delta x^{(k)} = \left| \mathbf{x}^{(k-1)} - \mathbf{x}^{(k)} \right|$ la magnitud del cambio en $\mathbf{x}$ durante la $k$-ésima iteración (realizada sin relajación, es decir, con $\omega = 1$). Si $k$ es suficientemente grande (se suele tomar $k \ge 5$), se puede demostrar que una aproximación del valor óptimo de $\omega$ es

$$
\omega_{\mathrm{opt}} \approx \frac{2}{1+\sqrt{1-\left(\Delta x^{(k+p)} / \Delta x^{(k)}\right)^{1 / p}}}
$$
 
donde $p$ es un número entero positivo.

De esta forma, el procedimiento habitual con el algoritmo Gauss-Seidel con relajación es el siguiente:
1. Calcula $k$ iteraciones con $\omega=1$ ($k=10$ es razonable).
2. Almacena $\Delta x^{(k)}$.
3. Realiza $p$ iteraciones más.
4. Almacena $\Delta x^{(k+p)}$.
5. Calcula $\omega_{\text {opt }}$.
6. Realiza todas las siguientes iteraciones con $\omega=\omega_{\text {opt }}$.

**Ejercicio 1 -** Programa una función ```gauss_seidel``` que implemente el método Gauss-Seidel con relajación. Calcula automáticamente $\omega_{opt}$ según se ha explicado anteriormente utilizando $k = 10$ y $p = 1$. La función debe devolver el vector solución, el número de iteraciones realizadas y el valor de $\omega_{opt}$ utilizado.

In [7]:
import numpy as np

def iter_esq(A, x, b, omega):

    n = len(b)
    x_old = np.copy(x)

    for i in range(n):
        x[i] = b[i] 

        for j in range(n):
            if j == i:
                continue # si i = j se salta esta iteracion
            x[i] -= A[i, j] * x[j]

        x[i] *= omega / A[i, i]

        x[i] += (1 - omega) * x_old[i]
        
    return x



def solve_gauss_seidel(
        A: np.ndarray,
        x: np.ndarray,
        b: np.ndarray,
        omega: float,
        tol: float,
        max_n_ite: int) -> np.ndarray :
    
    for i in range(1, max_n_ite):

        # guardo el candidato actual para comparar
        x_old = np.copy(x)

        # aaplicar el sistema iterativo de gauss seidel

        x = iter_esq(A, x, b, omega)

        #comprobar lo de la tolerancia

        dx = np.linalg.norm(x - x_old) # normal euclidea
        
        error = dx / np.linalg.norm(x)


        if error < tol:

            return x, i, omega
        
    print(f" cuidadillo: la tolerancia objetivo no ha sido alcanzada. la solucion actual tiene un error de {error}")

    return x, i, omega
    


In [9]:
A = np.array([[4.0, -2.0, 1.0],
             [-2.0, 4.0, -2.0],
             [1.0, -2.0, 4.0]], dtype = float)

b = np.array([11, -16, 17])

x = np.array([0.0, 0.0, 0.0])
omega = 0.7
tol = 10 ** -6
max_ite = 1000

solution = solve_gauss_seidel(A, x, b, omega, tol, max_ite)
solution

(array([ 0.99999691, -2.00000353,  2.99999866]), 27, 0.7)

In [14]:
import numpy as np

def iter_esq(A, x, b, omega):

    n = len(b)
    x_old = np.copy(x)

    for i in range(n):
        x[i] = b[i] 

        for j in range(n):
            if j == i:
                continue # si i = j se salta esta iteracion
            x[i] -= A[i, j] * x[j]

        x[i] *= omega / A[i, i]

        x[i] += (1 - omega) * x_old[i]

    return x

def solve_gauss_seidel_relax(
        A: np.ndarray,
        x: np.ndarray,
        b: np.ndarray,
        tol: float,
        max_n_ite: int) -> np.ndarray :
    
    omega = 1
    k = 10
    p = 1

    for i in range(1, max_n_ite):

        # guardo el candidato actual para comparar
        x_old = np.copy(x)

        # aaplicar el sistema iterativo de gauss seidel

        x = iter_esq(A, x, b, omega)

        #comprobar lo de la tolerancia

        dx = np.linalg.norm(x - x_old) # normal euclidea
        
        error = dx / np.linalg.norm(x)


        if error < tol:

            return x, i, omega
        
        if i == k:
            dx1 = dx


        if i == (k+p):
            dx2 = dx
            omega = 2 / (1 + (1 - (dx2 / dx1) ** (1 / p)) ** 0.5)
        
    print(f" cuidadillo: la tolerancia objetivo no ha sido alcanzada. la solucion actual tiene un error de {error}")

    return x, i, omega
    


In [17]:
A = np.array([[4.0, -2.0, 1.0],
             [-2.0, 4.0, -2.0],
             [1.0, -2.0, 4.0]], dtype = float)

b = np.array([11, -16, 17])

x = np.array([0.0, 0.0, 0.0])
tol = 10 ** -6
max_ite = 1000

solution = solve_gauss_seidel_relax(A, x, b, tol, max_ite)
solution

(array([ 0.99999998, -1.99999994,  3.00000003]),
 14,
 np.float64(1.0835680928076241))

In [ ]:
# implementacion para sistemas concretos

#enunciado problema 27 de la hoja

import numpy as np

def iter_esq(A, x, b, omega):

    n = len(b)
    x_old = np.copy(x)


    x[0] = omega * (x[1] - x[n-1]) / 4.0 + (1.0 - omega) * x[0]

    for i in range(1, n-1): # desde 0 no porue ya lo dfines a parte
        x[i] = omega * (x[i-1] x[i+1]) / 4.0 + (1.0 - omega) * x[0]
    
    x[n - 1] = omega * (100 - x[0] - x[n-2]) / 4.0 + (1.0 - omega) * x[n-1]

    return x

def solve_gauss_seidel_relax_sistema_J(
        # A: np.ndarray,
        x: np.ndarray,
        b: np.ndarray,
        tol: float,
        max_n_ite: int) -> np.ndarray :
    
    omega = 1
    k = 10
    p = 1

    for i in range(1, max_n_ite):

        # guardo el candidato actual para comparar
        x_old = np.copy(x)

        # aaplicar el sistema iterativo de gauss seidel

        x = iter_esq(A, x, b, omega)

        #comprobar lo de la tolerancia

        dx = np.linalg.norm(x - x_old) # normal euclidea
        
        error = dx / np.linalg.norm(x)


        if error < tol:

            return x, i, omega
        
        if i == k:
            dx1 = dx


        if i == (k+p):
            dx2 = dx
            omega = 2 / (1 + (1 - (dx2 / dx1) ** (1 / p)) ** 0.5)
        
    print(f" cuidadillo: la tolerancia objetivo no ha sido alcanzada. la solucion actual tiene un error de {error}")

    return x, i, omega
    


In [5]:
import numpy as np

n = 100

x = np.zeros(10)
x



array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

**Ejercicio 2 -** Resuelve el siguiente sistema de ecuaciones 
$$
\left[\begin{array}{rrr}
4 & -1 & 1 \\
-1 & 4 & -2 \\
1 & -2 & 4
\end{array}\right]\left[\begin{array}{l}
x_1 \\
x_2 \\
x_3
\end{array}\right]=\left[\begin{array}{r}
12 \\
-1 \\
5
\end{array}\right]
$$
mediante el método de Gauss-Seidel sin relajación, operando a mano.


**Ejercicio 3 -** Escribe un programa para resolver el siguiente sistema de $n$ ecuaciones por el método de Gauss-Seidel con relajación (el programa debe funcionar con cualquier valor de $n$):
$$
\left[\begin{array}{rrrrrrrrr}
2 & -1 & 0 & 0 & \ldots & 0 & 0 & 0 & 1 \\
-1 & 2 & -1 & 0 & \ldots & 0 & 0 & 0 & 0 \\
0 & -1 & 2 & -1 & \ldots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \ldots & -1 & 2 & -1 & 0 \\
0 & 0 & 0 & 0 & \ldots & 0 & -1 & 2 & -1 \\
1 & 0 & 0 & 0 & \ldots & 0 & 0 & -1 & 2
\end{array}\right]\left[\begin{array}{c}
x_1 \\
x_2 \\
x_3 \\
\vdots \\
x_{n-2} \\
x_{n-1} \\
x_n
\end{array}\right]=\left[\begin{array}{c}
0 \\
0 \\
0 \\
\vdots \\
0 \\
0 \\
1
\end{array}\right]
$$
Puedes reutilizar código anterior (Ejercicio 1). Ejecuta el programa con $n = 20$. Se puede demostrar que la solución exacta es $x_i = -n/4 +i/2, i = 1, 2, \dots , n$.

## Método del gradiente conjugado

Consideremos el problema de encontrar el vector $\mathbf{x}$ que minimiza la función escalar siguiente:

$$
f(\mathbf{x})=\frac{1}{2} \mathbf{x}^T \mathbf{A} \mathbf{x}-\mathbf{b}^T \mathbf{x}
$$

donde la matriz $\mathbf{A}$ es simétrica y definida positiva. Como $f(\mathbf{x})$ se minimiza cuando su gradiente $\nabla f = \mathbf{A} \mathbf{x}-\mathbf{b}$ es cero, vemos que esta minimización es equivalente a resolver:

$$
\mathbf{A} \mathbf{x} = \mathbf{b}
$$

Los métodos del gradiente logran esta minimización iterando desde un vector inicial $x_0$. En cada iteración $k$ se calcula una solución refinada

$$
\mathbf{x}_{k+1}=\mathbf{x}_k+\alpha_k \mathbf{s}_k
$$


La *longitud de paso* $\alpha_k$ se elige de forma que $\mathbf{x}_{k+1}$ minimice $f(\mathbf{x}_{k+1})$ en la *dirección de búsqueda* $\mathbf{s}_k$. Es decir, $\mathbf{x}_{k+1}$ debe satisfacer el sistema de ecuaciones a resolver:
$$
\mathbf{A}\left(\mathbf{x}_k+\alpha_k \mathbf{s}_k\right)=\mathbf{b}
$$

Ahora, introducimos el concepto de *residuo*

$$
\mathbf{r}_k=\mathbf{b}-\mathbf{A x}_k
$$
por lo que la ecuación anterior se puede poner como $\alpha \mathbf{A} \mathbf{s}_k=\mathbf{r}_k$. Pre-multiplicando ambos lados por $\mathbf{s}_k^T$ y resolviendo para $\alpha_k$, tenemos
$$
\alpha_k=\frac{\mathbf{s}_k^T \mathbf{r}_k}{\mathbf{s}_k^T \mathbf{A} \mathbf{s}_k}
$$
 
Todavía nos queda el problema de determinar la dirección de búsqueda $\mathbf{s}_k$. La intuición nos dice que elijamos $\mathbf{s}_k=-\nabla f=\mathbf{r}_k$, porque esta es la dirección del mayor cambio negativo en $f(x)$. El procedimiento resultante se conoce como el *método del descenso más pronunciado*. No es un algoritmo popular porque su convergencia puede ser lenta. El método del gradiente conjugado, más eficiente, utiliza la dirección de búsqueda

$$
\mathbf{s}_{k+1}=\mathbf{r}_{k+1}+\beta_k \mathbf{s}_k
$$

La constante $\beta_k$ se elige para que las dos direcciones de búsqueda sucesivas sean conjugadas
entre sí, lo que significa que 

$$
\mathbf{s}_{k+1}^T \mathbf{A} \mathbf{s}_k=0
$$
 
El gran atractivo de los gradientes conjugados es que la minimización en una dirección conjugada no va en contra de las minimizaciones anteriores (las minimizaciones no interfieren entre sí).
Usando este $s_{k+1}$ se obtiene que
 
$$
\left(\mathbf{r}_{k+1}^T+\beta_k \mathbf{s}_k^T\right) \mathbf{A} \mathbf{s}_k=0
$$

que da como resultado
 
$$
\beta_k=-\frac{\mathbf{r}_{k+1}^T \mathbf{A} \mathbf{s}_k}{\mathbf{s}_k^T \mathbf{A} \mathbf{s}_k}
$$ 

A continuación se muestra el pseudocódigo del algoritmo de gradiente conjugado:

1. Elegir $\mathbf{x}_0$ (arbitrariamente).
2. Sea $\mathbf{r}_0 \leftarrow \mathbf{b}-\mathbf{A x}_0$.
3. Sea $\mathbf{s}_0 \leftarrow \mathbf{r}_0$ (si se carece de una dirección de búsqueda previa, elija la dirección de gradiente más pronunciada).
4. hacer para $k=0,1,2, \ldots$ :
$$
\begin{aligned}
&\alpha_k \leftarrow \frac{\mathbf{s}_k^T \mathbf{r}_k}{\mathbf{s}_k^T \mathbf{A} \mathbf{s}_k} \text {. } \\
&\mathbf{x}_{k+1} \leftarrow \mathbf{x}_k+\alpha_k \mathbf{s}_k . \\
&\mathbf{r}_{k+1} \leftarrow \mathbf{b}-\mathbf{A x}_{k+1} \text {. } \\
&\text { si }\left|\mathbf{r}_{k+1}\right| \leq \varepsilon \text { exit loop (} \varepsilon \text { es la tolerancia del error). } \\
&\beta_k \leftarrow-\frac{\mathbf{r}_{k+1}^T \mathbf{A} \mathbf{s}_k}{\mathbf{s}_k^T \mathbf{A} \mathbf{s}_k} \text {. } \\
&\mathbf{s}_{k+1} \leftarrow \mathbf{r}_{k+1}+\beta_k \mathbf{s}_k . \\
&
\end{aligned}
$$


Se puede demostrar que los vectores residuo $\mathbf{r}_1, \mathbf{r}_2, \mathbf{r}_3, \dots$ producidos por el algoritmo son mutuamente ortogonales; es decir, $\mathbf{r}_i \cdot \mathbf{r}_j=0, i \neq j$. Supongamos ahora que hemos realizado suficientes iteraciones hasta haber calculado $n$ vectores residuo (recuerda que $n$ es la dimensión del problema). Ocurre entonces que el residuo resultante de la siguiente iteración será un vector nulo ($\mathbf{r}_{n+1} = \mathbf{0}$), lo que indica que se ha obtenido la solución exacta. Por lo tanto, parece que el algoritmo del gradiente conjugado
no es un método iterativo, porque alcanza efectivamente la solución después de $n$ iteraciones. Lo que suele ocurrir en la práctica es que la convergencia se alcanza en menos de $n$ iteraciones.

El método del gradiente conjugado no puede competir con los métodos directos en la resolución de pequeños sistemas de ecuaciones. Su fuerza reside en el manejo de sistemas grandes y dispersos (donde la mayoría de los elementos de $\mathbf{A}$ son cero). Es importante señalar que $\mathbf{A}$ sólo aparece en el algoritmo a través de su multiplicación por un vector; es decir, en la forma $\mathbf{Av}$, donde $\mathbf{v}$ es un vector (ya sea $\mathbf{s}_{k+1}$ o $\mathbf{s}_{k}$ ). Si $\mathbf{A}$ es dispersa, es posible escribir una función eficiente para la multiplicación de $\mathbf{A}$ por un vector y pasarla, en lugar de $A$, al algoritmo del gradiente conjugado.

**Ejercicio 4 -** Escribe una función ```conj_grad``` que implemente el algoritmo del gradiente conjugado. El número máximo de iteraciones permitido se fija en $n$ (el número de incógnitas). Dicha función debe internamente llamar a otra llamada ```av``` que devuelve el producto $\mathbf{Av}$. Esta función debe ser aportada por el usuario. El usuario también debe aportada el vector inicial $\mathbf{x}_0$ y el vector de términos independientes $\mathbf{b}$. La función devuelve el vector solución $\mathbf{x}$ y el número de iteraciones realizadas.

**Ejercicio 5 -** Resuelve el siguiente sistema de ecuaciones 
$$
\left[\begin{array}{rrr}
4 & -1 & 1 \\
-1 & 4 & -2 \\
1 & -2 & 4
\end{array}\right]\left[\begin{array}{l}
x_1 \\
x_2 \\
x_3
\end{array}\right]=\left[\begin{array}{r}
12 \\
-1 \\
5
\end{array}\right]
$$
mediante el método del gradiente conjugado, operando a mano.

**Ejercicio 6 -** Escribe un programa de ordenador para resolver el siguiente sistema de $n$ ecuaciones por el método del gradiente conjugado (el programa debe funcionar con cualquier valor de $n$):
$$
\left[\begin{array}{rrrrrrrrr}
2 & -1 & 0 & 0 & \ldots & 0 & 0 & 0 & 1 \\
-1 & 2 & -1 & 0 & \ldots & 0 & 0 & 0 & 0 \\
0 & -1 & 2 & -1 & \ldots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \ldots & -1 & 2 & -1 & 0 \\
0 & 0 & 0 & 0 & \ldots & 0 & -1 & 2 & -1 \\
1 & 0 & 0 & 0 & \ldots & 0 & 0 & -1 & 2
\end{array}\right]\left[\begin{array}{c}
x_1 \\
x_2 \\
x_3 \\
\vdots \\
x_{n-2} \\
x_{n-1} \\
x_n
\end{array}\right]=\left[\begin{array}{c}
0 \\
0 \\
0 \\
\vdots \\
0 \\
0 \\
1
\end{array}\right]
$$
Puedes reutilizar código anterior (Ejercicio 4). Ejecuta el programa con $n = 20$. Se puede demostrar que la solución exacta es $x_i = -n/4 +i/2, i = 1, 2, \dots , n$.